**SPEECH EMOTION RECOGNITION USING MLP CLASSIFIER**



[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
import os
Root = "/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24"
os.chdir(Root)

In [5]:
ls

Actor_01/  Actor_06/  Actor_11/  Actor_16/  Actor_21/
Actor_02/  Actor_07/  Actor_12/  Actor_17/  Actor_22/
Actor_03/  Actor_08/  Actor_13/  Actor_18/  Actor_23/
Actor_04/  Actor_09/  Actor_14/  Actor_19/  Actor_24/
Actor_05/  Actor_10/  Actor_15/  Actor_20/  modelForPrediction1.sav


In [6]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [7]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [8]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [9]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        print(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [10]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-01-01-01-02-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-01-01-01-01-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-03-01-01-01-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-02-02-01-02-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-02-02-01-01-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-02-01-02-01-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-02-01-02-02-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-02-02-02-02-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-01-01-02-01-16.wav
/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_16/03-01-02-01-01-02-16.wav
/content/d

In [11]:
x_train

array([[-6.07052612e+02,  4.73690758e+01, -5.25974560e+00, ...,
         1.22634958e-06,  8.76993766e-08,  7.94097677e-09],
       [-4.07671722e+02,  3.63255768e+01, -4.81901398e+01, ...,
         2.66403804e-05,  2.55928853e-06,  1.47395376e-08],
       [-6.58306274e+02,  7.80634842e+01, -7.86963344e-01, ...,
         1.97105035e-08,  6.83020884e-09,  6.12322193e-09],
       ...,
       [-7.71623718e+02,  8.32829742e+01,  9.73156643e+00, ...,
         1.47338275e-08,  6.55937393e-09,  5.52322010e-09],
       [-5.46402161e+02,  7.29353943e+01, -1.26167860e+01, ...,
         8.34204400e-07,  6.03438366e-08,  8.81847750e-09],
       [-5.49739441e+02,  5.59908600e+01, -1.27641907e+01, ...,
         9.06137529e-07,  7.62817294e-08,  7.99105404e-09]])

In [12]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(572, 191)


In [13]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [14]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [15]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [16]:
#Predict for the test set
y_pred=model.predict(x_test)

In [17]:
y_pred

array(['disgust', 'calm', 'happy', 'calm', 'calm', 'fearful', 'fearful',
       'fearful', 'happy', 'happy', 'happy', 'fearful', 'calm', 'happy',
       'calm', 'happy', 'happy', 'disgust', 'fearful', 'happy', 'calm',
       'calm', 'fearful', 'calm', 'fearful', 'happy', 'calm', 'fearful',
       'calm', 'calm', 'calm', 'happy', 'happy', 'fearful', 'fearful',
       'fearful', 'calm', 'happy', 'calm', 'fearful', 'calm', 'happy',
       'happy', 'fearful', 'fearful', 'calm', 'calm', 'calm', 'calm',
       'fearful', 'happy', 'calm', 'fearful', 'calm', 'calm', 'calm',
       'fearful', 'calm', 'calm', 'happy', 'fearful', 'fearful',
       'fearful', 'happy', 'fearful', 'happy', 'calm', 'calm', 'happy',
       'happy', 'disgust', 'calm', 'fearful', 'disgust', 'happy',
       'fearful', 'fearful', 'happy', 'calm', 'fearful', 'disgust',
       'happy', 'calm', 'calm', 'calm', 'fearful', 'calm', 'calm', 'calm',
       'calm', 'happy', 'fearful', 'calm', 'fearful', 'happy', 'fearful',
       

In [18]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 64.92%


In [19]:
from sklearn.metrics import accuracy_score, f1_score

In [20]:
f1_score(y_test, y_pred,average=None)

array([0.72580645, 0.30769231, 0.79545455, 0.64761905])

In [21]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,calm,calm
2,happy,happy
3,calm,calm
4,disgust,calm
5,fearful,fearful
6,fearful,fearful
7,happy,fearful
8,disgust,happy
9,happy,happy


In [22]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [23]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Colab Notebooks/Audio_Speech_Actors_01-24/Actor_03/03-01-01-01-01-01-03.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['happy'], dtype='<U7')

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
feature

array([[-6.50819458e+02,  7.50591049e+01, -1.55845547e+00,
         1.06763639e+01,  3.44822931e+00,  9.05338001e+00,
        -3.50779057e+00, -5.24679756e+00, -8.94363594e+00,
        -2.10030413e+00, -1.35191846e+00, -7.44291639e+00,
        -1.81947124e+00, -6.55488521e-02, -4.60928965e+00,
        -1.80607939e+00, -1.34068501e+00, -3.46330714e+00,
        -2.94180703e+00, -2.03767347e+00, -3.06777334e+00,
        -2.72727633e+00, -2.39168406e-01, -3.70551920e+00,
        -3.82644176e+00, -2.54342937e+00, -4.75811005e+00,
        -2.09809232e+00, -3.18546438e+00, -3.15821862e+00,
        -2.00466394e+00, -1.14599335e+00, -1.45911217e+00,
        -2.32801986e+00, -1.67657316e+00, -2.17620564e+00,
        -1.41877639e+00, -8.81222367e-01, -2.31366205e+00,
        -1.24511921e+00,  6.37808800e-01,  6.33684993e-01,
         6.47369683e-01,  6.60584629e-01,  7.10182190e-01,
         6.67255223e-01,  6.23698890e-01,  6.21280432e-01,
         6.50761962e-01,  6.30603492e-01,  6.39755666e-0